In [56]:
class CarInfo:
    def __init__(self, id, model_name: str, body_type: str, fuel_type: str, price: int, power: int,
                 fuel_efficiency: str, model_year: int, size: int, engine_type: str, image_link: str, brand: str):
        self.__id = id
        self.__model_name = model_name
        self.__body_type = body_type
        self.__fuel_type = fuel_type
        self.__price = price
        self.__power = power
        self.__fuel_efficiency = fuel_efficiency  # 할당 누락 수정
        self.__model_year = model_year
        self.__size = size
        self.__engine_type = engine_type
        self.__image_link = image_link
        self.__brand = brand

    @property
    def id(self):
        return self.__id

    @property
    def model_name(self):
        return self.__model_name

    @property
    def body_type(self):
        return self.__body_type

    @property
    def fuel_type(self):
        return self.__fuel_type

    @property
    def price(self):
        return self.__price

    @property
    def power(self):
        return self.__power

    @property
    def fuel_efficiency(self):
        return self.__fuel_efficiency

    @property
    def model_year(self):
        return self.__model_year

    @property
    def size(self):
        return self.__size

    @property
    def engine_type(self):
        return self.__engine_type

    @property
    def image_link(self):
        return self.__image_link

    @property
    def brand(self):
        return self.__brand

    def __repr__(self):
        return (f'CarInfo({self.__id}, {self.__model_name}, {self.__body_type}, {self.__fuel_type}, {self.__price}, '
                f'{self.__power}, {self.__fuel_efficiency}, {self.__model_year}, {self.__size}, '
                f'{self.__engine_type}, {self.__image_link}, {self.__brand})')


# -------------------------------------------------------

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup as bs

# 드라이버 실행
driver = webdriver.Chrome()

# 웹페이지 접속
driver.get("https://naver.com")
time.sleep(3)

# 검색어 입력
search_box = driver.find_element(By.ID, "query")
search_box.send_keys("네이버 자동차")
search_box.send_keys(Keys.ENTER)
time.sleep(3)

# URL 수집
url_list = []

for page in range(1, 54):
    try:
        container = driver.find_element(By.XPATH, f'//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[3]/div[{page}]')
        car_list = container.find_elements(By.CSS_SELECTOR, ".info_box")

        for car_tag in car_list:
            link = car_tag.find_element(By.CSS_SELECTOR, "a:first-of-type")
            href = link.get_attribute('href')
            if href:
                url_list.append(href + '%20%EC%A0%95%EB%B3%B4')
    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

# 모델별 상세 페이지 데이터 수집
car_info_list = []
idx = 1

for url in url_list:
    try:
        driver.get(url)
        time.sleep(3)

        soup = bs(driver.page_source, 'html.parser')

        # 모델명
        model_name_tag = soup.select_one('span.area_text_title')
        model_name = model_name_tag.text.strip() if model_name_tag else None

        # 바디 타입
        body_type_tag = soup.select_one('span.txt:first-of-type')
        body_type = body_type_tag.text.strip() if body_type_tag else None

        # 연식
        model_year_tag = soup.select_one('div.sub_title span.txt:last-of-type')
        model_year = model_year_tag.text.strip() if model_year_tag else None

        # 스펙 영역
        spec_tags = soup.select('div.detail_info')
        if not spec_tags:
            continue
        info_groups = spec_tags[0].select('div.info_group')

        if len(info_groups) < 10:
            continue

        # 가격
        price_text = info_groups[0].text.strip()
        price_start = price_text.split('~')[0]
        price = int(price_start.replace(',', '').replace('만원', ''))

        # 연료타입
        fuel_type = info_groups[1].text.strip()

        # 연비
        fuel_efficiency_text = info_groups[2].text.strip()
        fuel_efficiency = fuel_efficiency_text.split('~')[0].replace('복합', '').replace('km/l', '').strip()

        # 파워
        power_text = info_groups[3].text.strip()
        power = power_text.split('/')[0].replace('hp', '').strip()
        power = int(power)

        # 엔진타입
        engine_type = info_groups[7].text.strip()

        # 사이즈 계산
        wheelbase_text = info_groups[-1].text.strip()
        wheelbase = int(''.join(filter(str.isdigit, wheelbase_text)))

        length_text = info_groups[9].text.strip()
        length = int(''.join(filter(str.isdigit, length_text)))

        size = round((wheelbase / length) * 100, 2)

        # 이미지 링크
        img_tag = soup.select_one('div.thumb img')
        image_link = img_tag['src'] if img_tag else None

        # 브랜드
        brand = model_name.split(' ')[0] if model_name else None

        car = CarInfo(
            id=idx,
            model_name=model_name,
            body_type=body_type,
            fuel_type=fuel_type,
            price=price,
            power=power,
            fuel_efficiency=fuel_efficiency,
            model_year=int(model_year.replace('년', '').strip()) if model_year else None,
            size=size,
            engine_type=engine_type,
            image_link=image_link,
            brand=brand
        )
        car_info_list.append(car)
        idx += 1

    except Exception as e:
        print(f"URL {url}에서 오류 발생: {e}")

driver.quit()

print(len(car_info_list))


페이지 3에서 오류 발생: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[3]/div[3]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000101246a54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x000000010123ecf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x0000000100d8a864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000100dd1410 cxxbridge1$string$len + 382736
4   chromedriver                        0x0000000100e12480 cxxbridge1$string$len + 649088
5   chromedriver                        0x0000000100dc57ec cxxbridge1$string$len + 334572
6   chromedriver                        0x000000010120bccc cxxbridge1$str$ptr + 2562928
7   chromedriver    

In [134]:
class CarInfo:
    def __init__(self, id, model_name: str, body_type: str, fuel_type: str, price: int, power: int,
                 fuel_efficiency: str, model_year: int, size: int, engine_type: str, image_link: str, brand: str):
        self.__id = id
        self.__model_name = model_name
        self.__body_type = body_type
        self.__fuel_type = fuel_type
        self.__price = price
        self.__power = power
        self.__fuel_efficiency = fuel_efficiency
        self.__model_year = model_year
        self.__size = size
        self.__engine_type = engine_type
        self.__image_link = image_link
        self.__brand = brand

    @property
    def id(self):
        return self.__id

    @property
    def model_name(self):
        return self.__model_name

    @property
    def body_type(self):
        return self.__body_type

    @property
    def fuel_type(self):
        return self.__fuel_type

    @property
    def price(self):
        return self.__price

    @property
    def power(self):
        return self.__power

    @property
    def fuel_efficiency(self):
        return self.__fuel_efficiency

    @property
    def model_year(self):
        return self.__model_year

    @property
    def size(self):
        return self.__size

    @property
    def engine_type(self):
        return self.__engine_type

    @property
    def image_link(self):
        return self.__image_link

    @property
    def brand(self):
        return self.__brand

    def __repr__(self):
        return (f'CarInfo({self.__id}, {self.__model_name}, {self.__body_type}, {self.__fuel_type}, {self.__price}, '
                f'{self.__power}, {self.__fuel_efficiency}, {self.__model_year}, {self.__size}, '
                f'{self.__engine_type}, {self.__image_link}, {self.__brand})')


# -------------------------------------------------------

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup as bs

# 드라이버 실행
driver = webdriver.Chrome()

# 웹페이지 접속
driver.get("https://naver.com")
time.sleep(3)

# 검색어 입력
search_box = driver.find_element(By.ID, "query")
search_box.send_keys("네이버 자동차")
search_box.send_keys(Keys.ENTER)
time.sleep(3)

# URL 수집
url_list = []

for page in range(1, 54):
    try:
        container = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[3]/div[' + str(page) + ']')
        car_list = container.find_elements(By.CSS_SELECTOR, ".info_box")

        for car_tag in car_list:
            link = car_tag.find_element(By.CSS_SELECTOR, "a:first-of-type")
            href = link.get_attribute('href')
            if href:
                url_list.append(href + '%20%EC%A0%95%EB%B3%B4')
        time.sleep(3)


        next_button = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[4]/div/a[2]')
        next_button.click()
        time.sleep(3)

    except Exception as e:
        print(f"페이지 {page}에서 오류 발생: {e}")

print(len(url_list))


페이지 53에서 오류 발생: Message: element click intercepted: Element <a nocr="" onclick="goOtherTCR(this, 'a=nco_x58*3.page&amp;r=1&amp;i=18000090_000000000000');return false;" href="#" class="pg_next" data-kgs-page-action-next="" aria-disabled="true">...</a> is not clickable at point (477, 563). Other element would receive the click: <div class="pgs">...</div>
  (Session info: chrome=135.0.7049.115)
Stacktrace:
0   chromedriver                        0x0000000100f1ea54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x0000000100f16cf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x0000000100a62864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000100aaef3c cxxbridge1$string$len + 406076
4   chromedriver                        0x0000000100aad4a8 cxxbridge1$string$len + 399272
5   chromedriver                        0x0000000100aab2bc cxxbridge1$string$len + 390588
6   chromedriver                        0x0000000100aaa6b8

In [128]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time

# 드라이버 실행
driver = webdriver.Chrome()

# 웹페이지 접속
driver.get("https://naver.com")
time.sleep(3)

# 검색어 입력
search_box = driver.find_element(By.ID, "query")
search_box.send_keys("네이버 자동차")
search_box.send_keys(Keys.ENTER)
time.sleep(3)

# 5천만원 이하 필터링
button = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[1]/div/div/div/ul/li[5]/div/a')
button.click()
time.sleep(2)

slider = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[1]/div/div/div/div/div/div/div[2]/div[1]/div[3]/div/span/span[7]')
slider.click()
time.sleep(2)

actions = ActionChains(driver)
for _ in range(2):
    actions.send_keys(Keys.ARROW_LEFT).perform()
time.sleep(2)

apply = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/a')
apply.click()
time.sleep(2)

# URL 수집
url_list = []

for i in range(1, 16):
    try:
        # ✅ 컨테이너 안에 info_box만 찾기
        container = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[3]/div[ '+ str(i) + ']')
        car_list = container.find_elements(By.CSS_SELECTOR, ".info_box")

        for car_tag in car_list:
            link = car_tag.find_element(By.CSS_SELECTOR, "a:first-of-type")
            href = link.get_attribute('href')
            if href:
                url_list.append(href + '%20%EC%A0%95%EB%B3%B4')

        next_button = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[4]/div/a[2]')
        next_button.click()
        time.sleep(3)

    except Exception as e:
        print(f"오류 발생: {e}")

# # 최종 결과 출력
# print(f"총 수집된 링크 수: {len(url_list)}개")
# print(url_list)

# 상세 정보 추출
car_info_list = []

for idx, url in enumerate(url_list, 1):
    try:
        driver.get(url)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span.area_text_title strong._text'))
        )

        soup = bs(driver.page_source, 'html.parser')

        # ✅ 모델명
        model_name_tag = soup.select_one('span.area_text_title strong._text')
        model_name = model_name_tag.text.strip() if model_name_tag else None

        # ✅ body_type & model_year
        sub_title_tags = soup.select('div.sub_title .txt')
        body_type = None
        model_year = None

        if len(sub_title_tags) > 1:
            body_type = sub_title_tags[0].text.strip()
            model_year = sub_title_tags[1].text.strip()

        # ✅ image_link (a 태그에서 onclick 속성 값 추출)
        image_tag = soup.select_one('div.detail_info a.thumb.type_87_87 img')
        image_link = image_tag['src'] if image_tag else None

        # ✅ info_group 내부 데이터 가져오기
        info_groups = soup.select('div.info .info_group')

        def get_text_from_dd(index):
            """ info_group 내부의 특정 dd 값을 가져오는 함수 """
            if len(info_groups) > index:
                dd_tag = info_groups[index].select_one('dd')
                return dd_tag.text.strip() if dd_tag else None
            return None

        def extract_first_number(text):
            """ 숫자만 추출해서 반환하는 함수 (예: '2,797~3,639만원' → 2797) """
            numbers = re.findall(r'\d[\d,]*', text)
            return int(numbers[0].replace(',', '')) if numbers else None

        # ✅ price (첫 번째 info_group에서 ~ 앞 숫자만)
        price_text = get_text_from_dd(0)
        price = extract_first_number(price_text) if price_text else None

        # ✅ fuel_type (두 번째 info_group)
        fuel_type = get_text_from_dd(1)

        # ✅ fuel_efficiency (세 번째 info_group에서 ~ 앞 숫자만)
        fuel_efficiency_text = get_text_from_dd(2)
        fuel_efficiency = extract_first_number(fuel_efficiency_text) if fuel_efficiency_text else None

        # ✅ power (네 번째 info_group에서 ~ 앞 숫자만)
        power_text = get_text_from_dd(3)
        power = extract_first_number(power_text) if power_text else None

        # ✅ engine_type (일곱 번째 info_group에서 dd 값 + span.value_text 값)
        engine_type = None
        if len(info_groups) > 6:
            engine_dd = info_groups[6].select_one('dd')
            engine_span = info_groups[6].select_one('span.value_text')
            engine_type = (engine_dd.text.strip() if engine_dd else '') + " " + (engine_span.text.strip() if engine_span else '')

        # ✅ size 계산 (10번째 / 13번째 * 100, 소수점 2자리 반올림)
        size = None
        size_text1 = get_text_from_dd(9)
        size_text2 = get_text_from_dd(12)

        if size_text1 and size_text2:
            size_num1 = extract_first_number(size_text1)
            size_num2 = extract_first_number(size_text2)
            if size_num1 and size_num2:
                size = round((size_num2 / size_num1) * 100, 2)

        # ✅ 브랜드 정보 (추후 추가)
        brand = None

        # ✅ CarInfo 객체 생성 및 저장
        if model_name:
            car_info = CarInfo(
                id=idx,
                model_name=model_name,
                body_type=body_type,
                fuel_type=fuel_type,
                price=price,
                power=power,
                fuel_efficiency=fuel_efficiency,
                model_year=model_year,
                size=size,
                engine_type=engine_type,
                image_link=image_link,
                brand=brand
            )
            car_info_list.append(car_info)
            print(f"[{idx}] {car_info}")
        else:
            print(f"[{idx}] 모델명 없음")

    except TimeoutException:
        print(f"[{idx}] 타임아웃 발생: {url}")
    except Exception as e:
        print(f"[{idx}] 에러 발생: {e} URL: {url}")

print("\n✅ 총 수집된 CarInfo 객체 수:", len(car_info_list))


오류 발생: Message: element click intercepted: Element <a nocr="" onclick="goOtherTCR(this, 'a=nco_x58*3.page&amp;r=1&amp;i=18000090_000000000000');return false;" href="#" class="pg_next" data-kgs-page-action-next="" aria-disabled="true">...</a> is not clickable at point (474, 489). Other element would receive the click: <div class="pgs">...</div>
  (Session info: chrome=135.0.7049.115)
Stacktrace:
0   chromedriver                        0x000000010103ea54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x0000000101036cf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x0000000100b82864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000100bcef3c cxxbridge1$string$len + 406076
4   chromedriver                        0x0000000100bcd4a8 cxxbridge1$string$len + 399272
5   chromedriver                        0x0000000100bcb2bc cxxbridge1$string$len + 390588
6   chromedriver                        0x0000000100bca6b8 cxxbridg